In [10]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [11]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [12]:
# 武汉市
roi = ee.FeatureCollection('users/311605001111/region/ChinaCity').filter(ee.Filter.eq('市','武汉市'))
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [13]:
# 移除无效像元
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = image.get('SOLAR_AZIMUTH_ANGLE')
    zenith = image.get('SOLAR_ZENITH_ANGLE')
    return image.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True))

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

# 计算研究区域内，影像面积
def ImageArea(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.set({'imagearea': imageareas.get('B2')})


In [14]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-31') \
    .map(maskL8sr) \
    .map(water_index)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

# 加载样本集,AWEI,threshold = 0

In [15]:
# 加载样本集,AWEI,threshold = 0
internal_awei0 = ee.FeatureCollection('users/311605001111/wuhan_2020')
# Map.addLayer(samples, {}, "WuhanTrainingPoints_2020")
print(internal_awei0.size().getInfo())
clt_water = internal_awei0.filter(ee.Filter.eq('waterclass',1))
print(clt_water.size().getInfo())
clt_nowater = internal_awei0.filter(ee.Filter.eq('waterclass',0))
print(clt_nowater.size().getInfo())

4952
1084
3868


# 加载样本集,区间均值

In [18]:
# 加载样本集,区间均值
internal_mean40 = ee.FeatureCollection('users/311605001111/region_mean/wuhan_2020')
# Map.addLayer(samples, {}, "WuhanTrainingPoints_2020")
print(internal_mean40.size().getInfo())
clt_water = internal_mean40.filter(ee.Filter.eq('waterclass',1))
print(clt_water.size().getInfo())
clt_nowater = internal_mean40.filter(ee.Filter.eq('waterclass',0))
print(clt_nowater.size().getInfo())

5950
644
5306


# k-mean均值过滤

In [19]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [20]:
filtered_mean40 = k_mean(internal_mean40)
print("筛选后，剩余的样本数： ",filtered_mean40.size().getInfo())
# a = filtered_mean40.filter(ee.Filter.eq('waterclass',1))
# print('水体样本的数目：',a.size().getInfo())
# a = filtered_mean40.filter(ee.Filter.eq('waterclass',0))
# print('陆地样本的数目：',a.size().getInfo())

筛选后，剩余的样本数：  5775


In [21]:
filtered_awei0 = k_mean(internal_awei0)
print("筛选后，剩余的样本数： ",filtered_awei0.size().getInfo())

筛选后，剩余的样本数：  4920


# 训练RF训练器

In [22]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# 分类标签
label = 'waterclass'


trainedClassifier_mean40_filter = ee.Classifier.smileRandomForest(20).train(filtered_mean40,label,bands)
trainedClassifier_mean40 = ee.Classifier.smileRandomForest(20).train(internal_mean40,label,bands)

trainedClassifier_awei0_filter = ee.Classifier.smileRandomForest(20).train(filtered_awei0,label,bands)
trainedClassifier_awei0 = ee.Classifier.smileRandomForest(20).train(internal_awei0,label,bands)

# 单景影像的验证

In [23]:
# sentinel-2
# 加载sentinel 图像
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)
sen2 = ee.Image('COPERNICUS/S2_SR/20200209T025851_20200209T030437_T50RKU')
Map.addLayer(maskS2clouds(sen2.clip(roi)),{'min': 0.0,'max': 0.3,'bands': ['B8', 'B4', 'B3']},'sen2')

sen9 = ee.Image('COPERNICUS/S2_SR/20201225T030131_20201225T030129_T50RKU')
Map.addLayer(maskS2clouds(sen9.clip(roi)),{'min': 0.0,'max': 0.3,'bands': ['B8', 'B4', 'B3']},'sen9')

In [24]:
image2 = maskL8sr(water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20200209')))
Map.addLayer(image2.clip(roi),visParams,'image2')

image9 = maskL8sr(water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20201225')))
Map.addLayer(image9.clip(roi),visParams,'image9')

In [26]:
classify2_awei0_filter = image2.clip(roi).select(bands).classify(trainedClassifier_awei0_filter).eq(1).rename('waterclass')
Map.addLayer(classify2_awei0_filter.selfMask(),{'palette':['cyan']}, 'classify2_awei0_filter')

classify2_awei0 = image2.clip(roi).select(bands).classify(trainedClassifier_awei0).eq(1).rename('waterclass')
Map.addLayer(classify2_awei0.selfMask(),{'palette':['blue']}, 'classify2_awei0')

classify2_mean40_filter = image2.clip(roi).select(bands).classify(trainedClassifier_mean40_filter).eq(1).rename('waterclass')
Map.addLayer(classify2_mean40_filter.selfMask(),{'palette':['yellow']}, 'classify2_mean40_filter')

classify2_mean40 = image2.clip(roi).select(bands).classify(trainedClassifier_mean40).eq(1).rename('waterclass')
Map.addLayer(classify2_mean40.selfMask(),{'palette':['green']}, 'classify2_mean40')

In [ ]:
classify9_awei0_filter = image9.clip(roi).select(bands).classify(trainedClassifier_awei0_filter).eq(1).rename('waterclass')
Map.addLayer(classify9_awei0_filter.selfMask(),{'palette':['cyan']}, 'classify9_awei0_filter')

classify9_awei0 = image9.clip(roi).select(bands).classify(trainedClassifier_awei0).eq(1).rename('waterclass')
Map.addLayer(classify9_awei0.selfMask(),{'palette':['blue']}, 'classify9_awei0')

classify9_mean40_filter = image9.clip(roi).select(bands).classify(trainedClassifier_mean40_filter).eq(1).rename('waterclass')
Map.addLayer(classify9_mean40_filter.selfMask(),{'palette':['yellow']}, 'classify9_mean40_filter')

classify9_mean40 = image9.clip(roi).select(bands).classify(trainedClassifier_mean40).eq(1).rename('waterclass')
Map.addLayer(classify9_mean40.selfMask(),{'palette':['green']}, 'classify9_mean40')

In [ ]:
city = ee.FeatureCollection('users/311605001111/ChinaCity')
Map.addLayer(city, {}, "ChinaCity")

Maryland = ee.Image('users/311605001111/Maryland_nationwide_2018').clipToCollection(city)
Map.addLayer(Maryland, {'palette':['white','blue'],'min':0,'max':100}, "Maryland2018")

from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [72, 17, 136, 55.5] 
vis = {'min':0, 'max':100}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(Maryland, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[10,10], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(Maryland)', fontsize=20)

In [ ]:
basemap = ee.Image.constant(0).clipToCollection(city).rename('waterClass')
jrc = ee.Image('JRC/GSW1_3/YearlyHistory/2018').remap([0,1,2,3],[0,0,1,2]).rename('waterClass').clipToCollection(city)
JRC = ee.ImageCollection([jrc,basemap]).sum()
Map.addLayer(JRC, {'palette':['ffffff', 'ffbbbb', '0000ff'],'min':0,'max':2}, "Jrc")

from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [72, 17, 136, 55.5] 
vis = {'palette':['ffffff', 'ffbbbb', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(JRC, region=region, vis_params=vis)
cartoee.add_colorbar(ax, vis,loc="right",label="water frequency", orientation="vertical",discrete=True)
cartoee.add_gridlines(ax, interval=[10,10], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(JRC)', fontsize=20)